### change working directory to root

In [56]:
import os

In [57]:
%pwd

'/home/adityasanyal1996/MLflow'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/adityasanyal1996/MLflow'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
data.shape

(1599, 12)

### Entity

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE : str
    unzip_data_dir: Path
    all_schema: dict

### configuration manager in src

In [11]:
from mlProject.constants import *
from mlProject.utils.common import create_directories, read_yaml 

In [12]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

### Components(data validation)

In [13]:
import os
from mlProject import logger

In [53]:
class DataValidation:
    def __init__(self, config : DataValidationConfig):
        self.config = config

    def validate_all_columns(self)->bool:
        try:
            validation_status = True

            #we want to erase the contents from the previous run
            with open(self.config.STATUS_FILE, 'w') as f:
                pass


            #all_schema_types = []
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema_keys = self.config.all_schema.keys()
            
            #for validating the data types
            size = len(all_schema_keys)
            count = 0
            for col in all_cols:            
                count += 1
                for key in list(all_schema_keys):
                    if (col == key):
                        if (str(data.dtypes[col]) != (self.config.all_schema)[key]):
                            validation_status  = False
                            logger.info(f"{col} type in dataset doesn't match with {key} in schema")
                            break

                if count == size:
                    pass
                else:
                    continue


            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Type Validation status: {validation_status}")

                f.write("\n")
            
            validation_status = True
            #for validating the columns
            for col in all_cols:
                if col not in all_schema_keys:
                    validation_status = False
                    logger.info(f"{col} column doesn't match with schema")

            
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Column Validation status: {validation_status}")
            

            

            return validation_status
        
        except Exception as e:
            raise e

### Pipeline

In [60]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-06-19 13:17:45,550: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-19 13:17:45,556: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-19 13:17:45,557: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-19 13:17:45,559: INFO: common: created directory at: artifacts]
[2024-06-19 13:17:45,561: INFO: common: created directory at: artifacts/data_validation]


In [16]:
employees = [('Stuti', 28, 'Varanasi', 20000.0), 
            ('Saumya', 32, 'Delhi', 25000.0), 
            ('Aditya', 25, 'Mumbai', 40000.0), 
            ('Saumya', 32, 'Delhi', 35000.0), 
            ('Saumya', 32, 'Delhi', 30000.0), 
            ('Saumya', 32, 'Mumbai', 20000.0), 
            ('Aditya', 40, 'Dehradun', 24000.0), 
            ('Seema', 32, 'Delhi', 70000.0) 
            ] 
  
# Create a DataFrame 
df = pd.DataFrame(employees,  
                  columns = ['Name', 'Age', 
                             'City', 'Salary'])
df

,Name,Age,City,Salary
0,Stuti,28,Varanasi,20000.0
1,Saumya,32,Delhi,25000.0
2,Aditya,25,Mumbai,40000.0
3,Saumya,32,Delhi,35000.0
4,Saumya,32,Delhi,30000.0
5,Saumya,32,Mumbai,20000.0
6,Aditya,40,Dehradun,24000.0
7,Seema,32,Delhi,70000.0


In [17]:
type(str(df.dtypes['Salary']))



str

In [18]:
test = list(df.columns)
#list(df.columns)
type(test[0])

str

In [19]:
type(test[0])

str

In [20]:
print(str('hello'))

hello


In [25]:
from mlProject.utils.common import read_yaml
from pathlib import Path

path = Path("schema.yaml")

file = read_yaml(path)

[2024-06-19 12:06:28,995: INFO: common: yaml file: schema.yaml loaded successfully]


In [22]:
file.COLUMNS.values()
string = []
for element in list(file.COLUMNS.values()):
    string.append(str(element))
print(string)

['float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'int64']


In [23]:
s = file.COLUMNS.values()
print(s)
for i in s:
    print(type(s))

dict_values(['float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'int64'])
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>
<class 'dict_values'>


In [24]:
'''type(str(df.dtypes['Salary']))

for col in cols:
    p = str(df.dtypes[col])
    print(p)
'''
cols = list(df.columns)
yml = list(file.COLUMNS.values())
for col in cols:
    if str(df.dtypes[col]) not in yml:
        print(f"{col} doesn't match")

Name doesn't match
City doesn't match


In [31]:
keys = file.COLUMNS.keys()
for i in keys:
    print(file.COLUMNS.i)

BoxKeyError: "'ConfigBox' object has no attribute 'i'"